In [31]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, ElasticNet
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [32]:
train = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\train.csv",index_col=0,parse_dates=['time'])
test = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\test.csv",index_col=0,parse_dates=['time'])

In [33]:
train['day'] = train['time'].dt.day
train['month'] = train['time'].dt.month
train['weekday'] = train['time'].dt.weekday
train['hour'] = train['time'].dt.hour

test['day'] = test['time'].dt.day
test['month'] = test['time'].dt.month
test['weekday'] = test['time'].dt.weekday
test['hour'] = test['time'].dt.hour

In [34]:
train['weekday']  =  train['weekday'].map(lambda x: 1 if ((x==5)|(x==6)) else 0)
test['weekday']  =  test['weekday'].map(lambda x: 1 if ((x==5)|(x==6)) else 0)

In [35]:
train = train.drop('time', axis=1)
test = test.drop('time', axis=1)

train_dum = pd.get_dummies(train,drop_first = True)
test_dum = pd.get_dummies(test,drop_first = True)

In [36]:
X_train = train_dum.drop("congestion",axis=1)
y_train = np.log1p(train_dum["congestion"])

In [37]:
##### LogisticRegression #######

In [38]:
lr = LinearRegression()
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
param = {}
gcv_lr = GridSearchCV(lr,cv = kfold, param_grid=param,scoring="neg_mean_absolute_error")
gcv_lr.fit(X_train,y_train)
print(gcv_lr.best_score_)

-0.2759282940518523


In [39]:
y_pred = np.expm1(gcv_lr.predict(test_dum))
submit = pd.DataFrame({"row_id":test.index,"congestion":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\linear.csv",index=False)

In [8]:
########### ElasticNet ##########

In [40]:
elastic = ElasticNet(random_state=24)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
param = {"alpha": np.linspace(0.001,5,10),"l1_ratio":np.linspace(0.1,0.9,9)}
gcv_elastic = GridSearchCV(elastic,cv = kfold, param_grid=param,scoring="neg_mean_absolute_error")
gcv_elastic.fit(X_train,y_train)
print(gcv_elastic.best_score_)
print(gcv_elastic.best_params_)

-0.2760833237380677
{'alpha': 0.001, 'l1_ratio': 0.1}


In [41]:
y_pred = np.expm1(gcv_elastic.predict(test_dum))
submit = pd.DataFrame({"row_id":test.index,"congestion":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\elastic.csv",index=False)

In [42]:
########## XGBoost #################

In [43]:
xgb = XGBRegressor(random_state=24)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
param = {"learning_rate":np.linspace(0.001,0.9,10),"max_depth":[None,3,4],"n_estimators":[25,50,100]}
gcv_xgb = GridSearchCV(xgb,cv = kfold, param_grid=param,scoring="neg_mean_absolute_error")
gcv_xgb.fit(X_train,y_train)
print(gcv_xgb.best_score_)
print(gcv_xgb.best_params_)

-0.15323471943564365
{'learning_rate': 0.7002222222222222, 'max_depth': None, 'n_estimators': 100}


In [44]:
y_pred = np.expm1(gcv_xgb.predict(test_dum))
submit = pd.DataFrame({"row_id":test.index,"congestion":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\xgb.csv",index=False)

In [45]:
############ CatBoostRegressor ###########

In [46]:
cat =  CatBoostRegressor(random_state=24)

cat.fit(X_train,y_train)

y_pred = np.expm1(cat.predict(test_dum))
submit = pd.DataFrame({"row_id":test.index,"congestion":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\cat.csv",index=False)

Learning rate set to 0.118832
0:	learn: 0.4034409	total: 204ms	remaining: 3m 23s
1:	learn: 0.3921150	total: 251ms	remaining: 2m 5s
2:	learn: 0.3838382	total: 296ms	remaining: 1m 38s
3:	learn: 0.3765966	total: 339ms	remaining: 1m 24s
4:	learn: 0.3698500	total: 387ms	remaining: 1m 16s
5:	learn: 0.3643660	total: 431ms	remaining: 1m 11s
6:	learn: 0.3591192	total: 478ms	remaining: 1m 7s
7:	learn: 0.3536846	total: 525ms	remaining: 1m 5s
8:	learn: 0.3501875	total: 568ms	remaining: 1m 2s
9:	learn: 0.3451872	total: 613ms	remaining: 1m
10:	learn: 0.3389924	total: 660ms	remaining: 59.3s
11:	learn: 0.3366823	total: 707ms	remaining: 58.2s
12:	learn: 0.3321796	total: 753ms	remaining: 57.2s
13:	learn: 0.3301759	total: 797ms	remaining: 56.1s
14:	learn: 0.3264137	total: 842ms	remaining: 55.3s
15:	learn: 0.3243729	total: 887ms	remaining: 54.6s
16:	learn: 0.3214748	total: 932ms	remaining: 53.9s
17:	learn: 0.3196086	total: 975ms	remaining: 53.2s
18:	learn: 0.3160659	total: 1.02s	remaining: 52.7s
19:	learn

In [47]:
###### LGBMRegressor ############

In [48]:
lgbm = LGBMRegressor(random_state=24)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
params = {"learning_rate":np.linspace(0.001,0.9,10),"max_depth":[None,2,3,4],"n_estimators":[25,50,100]}
gcv_lgbm = GridSearchCV(lgbm,cv = kfold, param_grid=param,scoring="neg_mean_absolute_error")
gcv_lgbm.fit(X_train,y_train)
print(gcv_lgbm.best_score_)
print(gcv_lgbm.best_params_)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 679068, number of used features: 13
[LightGBM] [Info] Start training from score 3.815684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 679068, number of used features: 13
[LightGBM] [Info] Start training from score 3.816173
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough,

In [49]:
y_pred = np.expm1(gcv_lgbm.predict(test_dum))
submit = pd.DataFrame({"row_id":test.index,"congestion":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Tabular Playground Series(Assign3)\lgbm.csv",index=False)